In [ ]:
import numpy as np, matplotlib.pyplot as plt, cv2, tqdm
%matplotlib inline

In [ ]:
from importlib import reload

In [ ]:
from utils import saveVideo, show, drawShape

In [ ]:
import laneFindingPipeline

In [ ]:
undistort = laneFindingPipeline.Undistorter()

In [ ]:
from glob import glob
undistort.fit(glob('camera_cal/*.jpg'))

In [ ]:
# Load video
fpath = 'project_video.mp4'
import skvideo.io
reader = skvideo.io.FFmpegReader(fpath)
frames = []
for (i, f) in enumerate(tqdm.tqdm_notebook(
    reader.nextFrame(), 
    total=reader.inputframenum,
)):
    frames.append(f)

In [ ]:
show(frames[-1]);

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(12, 3))
dist = cv2.imread('camera_cal/calibration3.jpg')
undist = undistort(dist)
show(dist, axes[0], r'distorted $\rightarrow$')
show(undist, axes[1], r'undistorted $\rightarrow$')
show(laneFindingPipeline.transformChessboard(undist), axes[2], 'perspective warped')
fig.savefig('undistorted.png')

Set parameters for perspective transform.

In [ ]:
reload(laneFindingPipeline)

In [ ]:
perspective = laneFindingPipeline.PerspectiveTransformer()

In [ ]:
dist = frames[500]
fig, axes = plt.subplots(ncols=3, figsize=(12, 3))
undist = undistort(dist)
show(dist, axes[0], r'distorted $\rightarrow$')
show(drawShape(undist, perspective.src), axes[1], r'undistorted $\rightarrow$')

show(drawShape(perspective(undist), perspective.dst), axes[2], 'perspective')
fig.savefig('undistorted-road.png')

In [ ]:
vid = saveVideo((
    drawShape(thresholds(perspective(undistort(frame)), color=True), perspective.dst)
    for frame in tqdm.tqdm_notebook(frames, unit='frame', desc='transforming')
), 'perspectiveTransformed.mp4', total=len(frames))
vid

In [ ]:
cmf = laneFindingPipeline.ConvolutionalMarkingFinder()

index = 0
img = thresholds(perspective(undistort(frames[index])))
cmf.show(img)

It looks like solid and dashed markings can be distinguished by their histograms. This might be something to investigate later.

In [ ]:
fig, ax = plt.subplots()
left, right = cmf(img)
for points, label in zip([left, right], ['left', 'right']):
    ax.hist(points[1, :], bins=64, alpha=.5, label=label)
ax.legend()
ax.set_xlabel('$y$')
ax.set_ylabel('count');

In [ ]:
left, right = cmf(img)
left = laneFindingPipeline.LaneMarking(left)
right = laneFindingPipeline.LaneMarking(right)

fig, ax = show(img)
left.show(ax)
right.show(ax)
ax.set_xlim(0, img.shape[1])
ax.set_ylim(img.shape[0], 0);

In [ ]:
laneFinder = laneFindingPipeline.LaneFinder(undistort=undistort)
laneFinder.show(frames[0]);

In [ ]:
laneFinder.show(frames[1]);

In [ ]:
import utils

laneFinder = laneFindingPipeline.LaneFinder(undistort)
fig, axes = plt.subplots(nrows=2)
def inputs():
    for frame in frames:
        laneFinder.show(frame, axes=axes)
        yield utils.fig2img(fig)
        for ax in axes:
            ax.clear()
vid = saveVideo(inputs(), 'laneFits.mp4', total=len(frames))
fig.clf()
del fig
vid

Measure FPS without plotting.

In [ ]:
%%time
laneFinder = laneFindingPipeline.LaneFinder(undistort)
for frame in tqdm.tqdm_notebook(frames[:100], unit='frame'):
    laneFinder(frame)